# CNES FTP Database

Code to work with CNES (Cadastro Nacional de Estabelecimentos de Saúde) directories and files inside DATASUS FTP



In [1]:
from pysus.ftp.databases.cnes import CNES
cnes = CNES()

In [2]:
cnes.metadata

{'long_name': 'Cadastro Nacional de Estabelecimentos de Saúde',
 'source': 'https://cnes.datasus.gov.br/',
 'description': 'O Cadastro Nacional de Estabelecimentos de Saúde (CNES) é o sistema de informação oficial de cadastramento de informações de todos os estabelecimentos de saúde no país, independentemente de sua natureza jurídica ou de integrarem o Sistema Único de Saúde (SUS). Trata-se do cadastro oficial do Ministério da Saúde (MS) no tocante à realidade da capacidade instalada e mão-de-obra assistencial de saúde no Brasil em estabelecimentos de saúde públicos ou privados, com convênio SUS ou não.'}

CNES FTP Database has lazy loading (also applied to Directories), therefore its content will require explict `load()` to be displayed:

In [3]:
cnes.content

2023-09-19 11:57:40.905 | INFO     | pysus.ftp:content:440 - content is not loaded, use `load()` to load default paths


[]

In [4]:
cnes.load() # Loads default CNES content (from cnes.paths)

CNES - Cadastro Nacional de Estabelecimentos de Saúde

In [5]:
# Default content for (loaded) CNES database 
cnes.content

[/dissemin/publicos/CNES/200508_/Dados/DC,
 /dissemin/publicos/CNES/200508_/Dados/EE,
 /dissemin/publicos/CNES/200508_/Dados/EF,
 /dissemin/publicos/CNES/200508_/Dados/EP,
 /dissemin/publicos/CNES/200508_/Dados/EQ,
 /dissemin/publicos/CNES/200508_/Dados/GM,
 /dissemin/publicos/CNES/200508_/Dados/HB,
 /dissemin/publicos/CNES/200508_/Dados/IN,
 /dissemin/publicos/CNES/200508_/Dados/LT,
 /dissemin/publicos/CNES/200508_/Dados/PF,
 /dissemin/publicos/CNES/200508_/Dados/RC,
 /dissemin/publicos/CNES/200508_/Dados/SR,
 /dissemin/publicos/CNES/200508_/Dados/ST]

## CNES Groups

In [6]:
cnes.groups

{'DC': 'Dados Complementares',
 'EE': 'Estabelecimento de Ensino',
 'EF': 'Estabelecimento Filantrópico',
 'EP': 'Equipes',
 'EQ': 'Equipamentos',
 'GM': 'Gestão e Metas',
 'HB': 'Habilitação',
 'IN': 'Incentivos',
 'LT': 'Leitos',
 'PF': 'Profissional',
 'RC': 'Regra Contratual',
 'SR': 'Serviço Especializado',
 'ST': 'Estabelecimentos'}

## Loading groups

To load specific groups into `cnes` content, it's possible to pass them in the `load()` method:

In [7]:
cnes.load("DC")
cnes.load(["ST", "SR"])
print(str(len(cnes.content)) + " files loaded")

17838 files loaded


## Filtering files

#### by group (required)

In [8]:
# Get files by group
len(cnes.get_files("SR"))

5940

In [9]:
# Multiple groups
len(cnes.get_files(["ST", "SR"]))

11887

#### by UF (state)

In [10]:
# Get files by UF from group
print("SR - São Paulo files: " + str(len( cnes.get_files("SR", uf="SP" ))))

SR - São Paulo files: 220


In [11]:
# Multiple UFs
len(cnes.get_files("SR", uf=["SP", "RJ"]))

440

#### by year

In [12]:
len(cnes.get_files("SR", year=2023)) # or 23

216

In [13]:
# Multiple Years
len(cnes.get_files("SR", year=[20, 21, 22, 23]))

1243

#### by month

In [14]:
len(cnes.get_files("SR", month=[1, 2, 3, 4])) # or single month

1945

### Combining filters

In [15]:
files = cnes.get_files("SR", uf="SP", year=2023, month=[1,2,3])
files

[SRSP2301.dbc, SRSP2302.dbc, SRSP2303.dbc]

## Describing Files

In [16]:
from pprint import pprint
for file in files:
    pprint(cnes.describe(file))

{'group': 'Serviço Especializado',
 'last_update': '2023-02-17 07:31AM',
 'month': 'Janeiro',
 'name': 'SRSP2301.dbc',
 'size': '1.6 MB',
 'uf': 'São Paulo',
 'year': 2023}
{'group': 'Serviço Especializado',
 'last_update': '2023-03-14 02:34PM',
 'month': 'Fevereiro',
 'name': 'SRSP2302.dbc',
 'size': '1.6 MB',
 'uf': 'São Paulo',
 'year': 2023}
{'group': 'Serviço Especializado',
 'last_update': '2023-04-17 07:34AM',
 'month': 'Março',
 'name': 'SRSP2303.dbc',
 'size': '1.6 MB',
 'uf': 'São Paulo',
 'year': 2023}


## Downloading Files

In [17]:
parquets = cnes.download(files)
parquets

SRSP2303.parquet: 100%|██████████| 81.4k/81.4k [00:05<00:00, 14.8kB/s]


[/home/bida/pysus/SRSP2301.parquet,
 /home/bida/pysus/SRSP2302.parquet,
 /home/bida/pysus/SRSP2303.parquet]

### Reading files

PySUS uses Parquets as output, use the method `to_dataframe()` to read the file as pandas DataFrame

In [18]:
parquets[0].to_dataframe()

,CNES,CODUFMUN,SERV_ESP,CLASS_SR,SRVUNICO,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,...,CNPJ_MAN,CARACTER,AMB_NSUS,AMB_SUS,HOSP_NSUS,HOSP_SUS,COMPETEN,CONTSRVU,CNESTERC,NAT_JUR
0,2790920,351450,145,008,,0206,,,,M,...,00000000000000,1,1,0,0,0,202301,,,2062
1,2790920,351450,145,009,,0206,,,,M,...,00000000000000,1,1,0,0,0,202301,,,2062
2,2790920,351450,145,010,,0206,,,,M,...,00000000000000,1,1,0,0,0,202301,,,2062
3,2790920,351450,145,013,,0206,,,,M,...,00000000000000,1,1,0,0,0,202301,,,2062
4,3025489,351450,111,001,111,0206,,,,M,...,46137485000160,1,0,1,0,0,202301,1,,1244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144830,0008885,354780,141,002,,,,,,M,...,46522942000130,1,0,1,0,0,202301,,,1244
144831,0008885,354780,144,001,144,,,,,M,...,46522942000130,1,0,1,0,0,202301,1,,1244
144832,0008885,354780,159,001,159,,,,,M,...,46522942000130,1,0,1,0,0,202301,1,,1244
144833,0008885,354780,159,004,,,,,,M,...,46522942000130,1,0,1,0,0,202301,,,1244
